In [1]:
import psycopg2

In [2]:
# connection credentials
host = os.environ["PGHOST"]
dbname = 'udacity'
user = os.environ["PGUSER"]
password = os.environ["PGPASSWORD"]
port = os.environ["PGPORT"]

In [3]:
# connecting to the database
try:
    conn = psycopg2.connect( host=host
                            ,port=port
                            ,dbname = dbname
                            ,user=user
                            ,password=password
                           )
    
except psycopg2.Error as error:
    print("Error: could not connect to Postgres Database")
    print(error)
    

In [4]:
# retrieving a cursor
try:
    cur = conn.cursor()
    
except psycopg2.Error as error:
    print("Error: could not get cursor to the Database")
    print(error)
    
# set "autocommit" to True
conn.set_session(autocommit=True)

In [5]:

music_store_create_table_statement = \
    """
        DROP TABLE IF EXISTS
            public.music_store
        ;
        
        CREATE TABLE
            public.music_store(
                 transaction_id SMALLINT
                ,customer_name TEXT
                ,cashier_name TEXT
                ,year INTEGER
                ,albums_purchased TEXT[]
            )
        ;
    """

music_store_insert_into_statement = \
    """
        INSERT INTO
            public.music_store(
                 transaction_id
                ,customer_name
                ,cashier_name
                ,year
                ,albums_purchased
                
            )
            
        VALUES(
             %s
            ,%s
            ,%s
            ,%s
            ,%s
        )
        ;
    """

#### create the "music_store" table

In [12]:

try:
    cur.execute(music_store_create_table_statement)
    
except psycopg2.Error as error:
    print("Error: issue creating table")
    print(error)

In [6]:
    """
        INSERT INTO
            public.music_store(
                 transaction_id
                ,customer_name
                ,cashier_name
                ,year
                ,albums_purchased
                
            )
            
        VALUES(
             %(transaction_id)s
            ,%(customer_name)s
            ,%(cashier_name)s
            ,%(year)s
            ,%(albums_purchased)s
        )
        ;
    """

    """
        INSERT INTO
            public.music_store(
                 transaction_id
                ,customer_name
                ,cashier_name
                ,year
                ,albums_purchased
                
            )
            
        VALUES(
             %s
            ,%s
            ,%s
            ,%s
            ,%s
        )
        ;
    """

'\n    INSERT INTO\n        public.music_store(\n             transaction_id\n            ,customer_name\n            ,cashier_name\n            ,year\n            ,albums_purchased\n            \n        )\n        \n    VALUES(\n         %s\n        ,%s\n        ,%s\n        ,%s\n        ,%s\n    )\n    ;\n'

#### a dictionary would work
if we weren't also inserting into an `ARRAY[]` type column.  
We solve this by using a list of tuples in the cell below.

In [46]:

first_insert = dict()

first_insert = (
     { "transaction_id": "1"
      ,"customer_name": "Amanda"
      ,"cashier_name": "Sam"
      ,"year": "2000"
      ,"albumns_purchased": '{'"Rubber Soul"', '"Let it Be"'}'
     }
    ,{"transaction_id": "2"
      ,"customer_name": "Toby"
      ,"cashier_name": "Sam"
      ,"year": "2000"
      ,"albumns_purchased": '{'"My Generation"'}'
     }
    ,{"transaction_id": "3"
      ,"customer_name": "Max"
      ,"cashier_name": "Bob"
      ,"year": "2018"
      ,"albumns_purchased": '{'"Meet the Beatles"', '"Help!"'}'
     }
)

In [60]:

tuple_list = [
     (1,"Amanda","Sam",2000,'{"Rubber Soul","Let it Be"}')
    ,(2,"Toby","Sam",2000,'{"My Generation"}')
    ,(3,"Max","Bob",2018,'{"Meet the Beatles","Help!"}')
]

In [61]:

cur.executemany(music_store_insert_into_statement, tuple_list)

#### SELECT data
from the `music_store` table we just created

In [8]:

try:
    cur.execute(
        """
            SELECT
                *
                
            FROM
                public.music_store
        """
    )
except psycopg2.Error as error:
    print("Error: select *")
    print(error)
    
row = cur.fetchone()

while row:
    print(row)
    row = cur.fetchone()
    

(1, 'Amanda', 'Sam', 2000, ['Rubber Soul', 'Let it Be'])
(2, 'Toby', 'Sam', 2000, ['My Generation'])
(3, 'Max', 'Bob', 2018, ['Meet the Beatles', 'Help!'])


#### move the `music_store` table to 1NF
by splitting the "albuns_purchased" attribute into individual rows:

In [9]:

create_table_music_store_1nf_statement = \
    """
        DROP TABLE IF EXISTS
            public.music_store_1nf
        ;
        
        CREATE TABLE
            public.music_store_1nf
                (
                 transaction_id SMALLINT
                ,customer_name TEXT
                ,cashier_name TEXT
                ,year INTEGER
                ,albums_purchased TEXT
                )
    """

try:
    cur.execute(create_table_music_store_1nf_statement)
    
except psycopg2.Error as error:
    print("Error creating table")
    print(error)

In [13]:

music_store_1nf_insert_into_statement = \
    """
        INSERT INTO
            public.music_store_1nf(
                 transaction_id
                ,customer_name
                ,cashier_name
                ,year
                ,albums_purchased
                
            )
            
        VALUES(
             %s
            ,%s
            ,%s
            ,%s
            ,%s
        )
        ;
    """

# a variable holding a list of tuples is created
music_store_1nf_data = [
     (1,"Amanda","Sam",2000,"Rubber Soul")
    ,(1,"Amanda","Sam",2000,"Let it Be")
    ,(2,"Toby","Sam",2000,"My Generation")
    ,(3,"Max","Bob",2018,"Meet the Beatles")
    ,(3,"Max","Bob",2018,"Help!")
]

#### insert data into the new table

In [14]:

try:
    cur.executemany(music_store_1nf_insert_into_statement, music_store_1nf_data)
except psycopg2.Error as error:
    print("Error when inserting data")
    print(error)

#### SELECT data
from the 1NF table

In [15]:

try:
    cur.execute(
        """
            SELECT
                *
                
            FROM
                public.music_store_1nf
        """
    )
except psycopg2.Error as error:
    print("Error: select *")
    print(error)
    
row = cur.fetchone()

while row:
    print(row)
    row = cur.fetchone()


(1, 'Amanda', 'Sam', 2000, 'Rubber Soul')
(1, 'Amanda', 'Sam', 2000, 'Let it Be')
(2, 'Toby', 'Sam', 2000, 'My Generation')
(3, 'Max', 'Bob', 2018, 'Meet the Beatles')
(3, 'Max', 'Bob', 2018, 'Help!')


#### Applying 2NF to our data model
by splitting the current table into two distinct tables: `transactions` and `albums_sold`

In [16]:

create_table_transactions_statement = """
    DROP TABLE IF EXISTS
        public.transactions
    ;
    
    CREATE TABLE
        public.transactions
            (
             transaction_id SMALLINT
            ,customer_name TEXT
            ,cashier_name TEXT
            ,year INTEGER
            )
"""

create_table_albums_sold_statement = """
    DROP TABLE IF EXISTS
        public.albums_sold
    ;
    
    CREATE TABLE
        public.albums_sold
            (
             album_id SMALLINT
            ,transaction_id SMALLINT
            ,album_name TEXT
            )
"""

#### create the two new tables

In [18]:

create_statements_execution_list = [
     create_table_transactions_statement
    ,create_table_albums_sold_statement
]

for statement in create_statements_execution_list:

    try:
        cur.execute(statement)

    except psycopg2.Error as error:
        print("Error creating table")
        print(error)

In [36]:

transactions_data = [
     (1,"Amanda","Sam",2000)
    ,(2,"Toby","Sam",2000)
    ,(3,"Max","Bob",2018)
]


insert_into_transaction_statement = """

    INSERT INTO
        public.transactions(
             transaction_id
            ,customer_name
            ,cashier_name
            ,year
        )
        
    VALUES(
         %s
        ,%s
        ,%s
        ,%s
    )
    ;
"""


albums_sold_data = [
     (1,1,"Rubber Soul")
    ,(2,1,"Let it Be")
    ,(3,2,"My Generation")
    ,(4,3,"Meet the Beatles")
    ,(5,3,"Help!")
]

insert_into_albums_sold_statement = """

    INSERT INTO
        public.albums_sold(
             album_id
            ,transaction_id
            ,album_name
        )
        
    VALUES(
         %s
        ,%s
        ,%s
    )
    ;
"""

In [37]:
# insert "Transactions" data
try:
    cur.executemany(insert_into_transaction_statement, transactions_data)
    
except psycopg2.Error as error:
    print("Error inserting data")
    print(error)

# insert "Albums Sold" data
try:
    cur.executemany(insert_into_albums_sold_statement, albums_sold_data)
    
except psycopg2.Error as error:
    print("Error inserting data")
    print(error)

In [38]:

transactions_and_albums_sold_join_statement = """
    SELECT
        *

    FROM
        transactions

    LEFT JOIN
        albums_sold

    ON
        transactions.transaction_id
        =
        albums_sold.transaction_id
"""

In [39]:

try:
    cur.execute(transactions_and_albums_sold_join_statement)
    
except psycopg2.Error as error:
    print("Error retrieving data")
    print(error)
    
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

(1, 'Amanda', 'Sam', 2000, 1, 1, 'Rubber Soul')
(1, 'Amanda', 'Sam', 2000, 2, 1, 'Let it Be')
(2, 'Toby', 'Sam', 2000, 3, 2, 'My Generation')
(3, 'Max', 'Bob', 2018, 4, 3, 'Meet the Beatles')
(3, 'Max', 'Bob', 2018, 5, 3, 'Help!')


#### appying 3NF to our data model
by creating a third table: `employees`   
This removes transient dependencies from the `transactions` table

In [40]:

create_table_transactions_3nf_statement = """
    DROP TABLE IF EXISTS
        public.transactions_3nf
    ;
    
    CREATE TABLE
        public.transactions_3nf
            (
             transaction_id SMALLINT
            ,customer_name TEXT
            ,employee_id SMALLINT
            ,year INTEGER
            )
"""

create_table_employees_statement = """
    DROP TABLE IF EXISTS
        public.employess
    ;
    
    CREATE TABLE
        public.employees(
             employee_id SMALLINT
            ,employee_name TEXT
        )
    ;
"""

In [41]:

statement_execution_list = [
     create_table_transactions_3nf_statement
    ,create_table_employees_statement
]

for statement in statement_execution_list:
    
    try:
        cur.execute(statement)
        
    except psycopg2.Error as error:
        print("Error creating table")
        print(error)

In [45]:

insert_into_transactions_3nf_statement = """

    INSERT INTO
        public.transactions_3nf(
             transaction_id
            ,customer_name
            ,employee_id
            ,year
        )
        
    VALUES(
         %s
        ,%s
        ,%s
        ,%s
    )
    ;
"""

transactions_3nf_data = [
     (1,"Amanda",1,2000)
    ,(2,"Toby",1,2000)
    ,(3,"Max",2,2018)
]

insert_into_employees_statement = """

    INSERT INTO
        public.employees(
             employee_id
            ,employee_name
        )
        
    VALUES(
         %s
        ,%s
    )
    ;
"""

employees_data = [
     (1,"Sam")
    ,(2,"Bob")
]

In [47]:

# insert "Transactions 3NF" data
try:
    cur.executemany(insert_into_transactions_3nf_statement, transactions_3nf_data)
    
except psycopg2.Error as error:
    print("Error inserting data")
    print(error)
    
# insert "Employees" data
try:
    cur.executemany(insert_into_employees_statement, employees_data)
    
except psycopg2.Error as error:
    print("Error inserting data")
    print(error)

In [49]:

all_tables_join_statement = """

    SELECT	
        *

    FROM
        transactions_3nf

    LEFT JOIN
        employees

    ON
        transactions_3nf.employee_id
        =
        employees.employee_id

    LEFT JOIN
        albums_sold

    ON
        transactions_3nf.transaction_id
        =
        albums_sold.transaction_id
    ;
"""

In [50]:

try:
    cur.execute(all_tables_join_statement)
    
except psycopg2.Error as error:
    print("Error retrieving data")
    print(error)
    
row = cur.fetchone()

while row:
    print(row)
    row = cur.fetchone()

(1, 'Amanda', 1, 2000, 1, 'Sam', 1, 1, 'Rubber Soul')
(1, 'Amanda', 1, 2000, 1, 'Sam', 2, 1, 'Let it Be')
(2, 'Toby', 1, 2000, 1, 'Sam', 3, 2, 'My Generation')
(3, 'Max', 2, 2018, 2, 'Bob', 4, 3, 'Meet the Beatles')
(3, 'Max', 2, 2018, 2, 'Bob', 5, 3, 'Help!')
